In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV , train_test_split , cross_val_score
from sklearn.metrics import classification_report , confusion_matrix


from sklearn.linear_model import LogisticRegression


from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
import os
import warnings
warnings.filterwarnings('ignore')

print(os.listdir("../input/"))

['sample_submission.csv', 'test.csv', 'data_description.txt', 'train.csv']


In [2]:
train = pd.read_csv('../input/train.csv' , index_col= 'Id')
test = pd.read_csv('../input/test.csv'  , index_col= 'Id')
train = train[train.GrLivArea < 4500]
train.reset_index(drop=True, inplace=True)
label = train[['SalePrice']]
train.drop('SalePrice' , axis = 1 , inplace=True)
train.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal


In [3]:
label.head(3)

,SalePrice
0,208500
1,181500
2,223500


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 79 columns):
MSSubClass       1458 non-null int64
MSZoning         1458 non-null object
LotFrontage      1199 non-null float64
LotArea          1458 non-null int64
Street           1458 non-null object
Alley            91 non-null object
LotShape         1458 non-null object
LandContour      1458 non-null object
Utilities        1458 non-null object
LotConfig        1458 non-null object
LandSlope        1458 non-null object
Neighborhood     1458 non-null object
Condition1       1458 non-null object
Condition2       1458 non-null object
BldgType         1458 non-null object
HouseStyle       1458 non-null object
OverallQual      1458 non-null int64
OverallCond      1458 non-null int64
YearBuilt        1458 non-null int64
YearRemodAdd     1458 non-null int64
RoofStyle        1458 non-null object
RoofMatl         1458 non-null object
Exterior1st      1458 non-null object
Exterior2nd      1458 non-

In [5]:
numerical_col = []
cat_col = []
for x in train.columns:
    if train[x].dtype == 'object':
        cat_col.append(x)
        print(x+': ' + str(len(train[x].unique())))
    else:
        numerical_col.append(x)
        
print('CAT col \n', cat_col)
print('Numerical col\n')
print(numerical_col)

MSZoning: 5
Street: 2
Alley: 3
LotShape: 4
LandContour: 4
Utilities: 2
LotConfig: 5
LandSlope: 3
Neighborhood: 25
Condition1: 9
Condition2: 8
BldgType: 5
HouseStyle: 8
RoofStyle: 6
RoofMatl: 7
Exterior1st: 15
Exterior2nd: 16
MasVnrType: 5
ExterQual: 4
ExterCond: 5
Foundation: 6
BsmtQual: 5
BsmtCond: 5
BsmtExposure: 5
BsmtFinType1: 7
BsmtFinType2: 7
Heating: 6
HeatingQC: 5
CentralAir: 2
Electrical: 6
KitchenQual: 4
Functional: 7
FireplaceQu: 6
GarageType: 7
GarageFinish: 4
GarageQual: 6
GarageCond: 6
PavedDrive: 3
PoolQC: 4
Fence: 5
MiscFeature: 5
SaleType: 9
SaleCondition: 6
CAT col 
 ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'Kitche

In [6]:
numerical_col.remove('MSSubClass')
cat_col.append('MSSubClass')

In [7]:
train_num = train[numerical_col]
train_num.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008


In [8]:
imputer = Imputer(missing_values='NaN' , strategy='median' , axis = 0)
imputer = imputer.fit(train_num)
train_num = imputer.transform(train_num)

In [9]:
test_num = imputer.transform(test[numerical_col])

In [10]:
print(train_num.shape)
print(test_num.shape)

(1458, 35)
(1459, 35)


In [11]:
X_train , X_test , y_train , y_test=  train_test_split(train_num , label , test_size= 0.2 , random_state=123)

In [12]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
scores = cross_val_score(clf, X_train, y_train, cv=5).mean()
scores

0.848983524694561

In [13]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=0.3, normalize=True)
scores = cross_val_score(clf, X_train, y_train, cv=5).mean()
scores

0.8490697977664062

In [14]:
from sklearn.linear_model import ElasticNet
clf = ElasticNet(alpha=1, l1_ratio=0.5, normalize=False)
scores = cross_val_score(clf, X_train, y_train, cv=5).mean()
scores

0.8393453001148077

In [15]:
import xgboost
clf=xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
colsample_bytree=1, max_depth=7)
scores = cross_val_score(clf, X_train, y_train, cv=5).mean()
scores

0.8876468842215086

In [16]:
train_cat = train[cat_col]
test_cat = test[cat_col]
print(train_cat.info())
print(test_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 44 columns):
MSZoning         1458 non-null object
Street           1458 non-null object
Alley            91 non-null object
LotShape         1458 non-null object
LandContour      1458 non-null object
Utilities        1458 non-null object
LotConfig        1458 non-null object
LandSlope        1458 non-null object
Neighborhood     1458 non-null object
Condition1       1458 non-null object
Condition2       1458 non-null object
BldgType         1458 non-null object
HouseStyle       1458 non-null object
RoofStyle        1458 non-null object
RoofMatl         1458 non-null object
Exterior1st      1458 non-null object
Exterior2nd      1458 non-null object
MasVnrType       1450 non-null object
ExterQual        1458 non-null object
ExterCond        1458 non-null object
Foundation       1458 non-null object
BsmtQual         1421 non-null object
BsmtCond         1421 non-null object
BsmtExposure     1420

In [17]:
dropp = ['MiscFeature' , 'PoolQC' , 'Fence' ,'Alley' ]
train_cat.drop(columns=dropp , axis=1, inplace=True)

In [18]:
train_cat = train_cat.astype('category')
print(train_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 40 columns):
MSZoning         1458 non-null category
Street           1458 non-null category
LotShape         1458 non-null category
LandContour      1458 non-null category
Utilities        1458 non-null category
LotConfig        1458 non-null category
LandSlope        1458 non-null category
Neighborhood     1458 non-null category
Condition1       1458 non-null category
Condition2       1458 non-null category
BldgType         1458 non-null category
HouseStyle       1458 non-null category
RoofStyle        1458 non-null category
RoofMatl         1458 non-null category
Exterior1st      1458 non-null category
Exterior2nd      1458 non-null category
MasVnrType       1450 non-null category
ExterQual        1458 non-null category
ExterCond        1458 non-null category
Foundation       1458 non-null category
BsmtQual         1421 non-null category
BsmtCond         1421 non-null category
BsmtExposure 

In [19]:
test_cat.drop(columns=dropp , axis=1, inplace=True)
test_cat = test_cat.astype('category')
test_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 40 columns):
MSZoning         1455 non-null category
Street           1459 non-null category
LotShape         1459 non-null category
LandContour      1459 non-null category
Utilities        1457 non-null category
LotConfig        1459 non-null category
LandSlope        1459 non-null category
Neighborhood     1459 non-null category
Condition1       1459 non-null category
Condition2       1459 non-null category
BldgType         1459 non-null category
HouseStyle       1459 non-null category
RoofStyle        1459 non-null category
RoofMatl         1459 non-null category
Exterior1st      1458 non-null category
Exterior2nd      1458 non-null category
MasVnrType       1443 non-null category
ExterQual        1459 non-null category
ExterCond        1459 non-null category
Foundation       1459 non-null category
BsmtQual         1415 non-null category
BsmtCond         1414 non-null category
BsmtExposu

In [20]:
most_freq = {}
for col in train_cat.columns:
    p = train_cat[col].mode()[0] 
    train_cat[col].fillna(p, inplace=True)
    most_freq[col] = p

In [21]:
for col in train_cat.columns:
    test_cat[col].fillna(most_freq[col], inplace=True)

In [22]:
print(test_cat.info())
print(train_cat.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 40 columns):
MSZoning         1459 non-null category
Street           1459 non-null category
LotShape         1459 non-null category
LandContour      1459 non-null category
Utilities        1459 non-null category
LotConfig        1459 non-null category
LandSlope        1459 non-null category
Neighborhood     1459 non-null category
Condition1       1459 non-null category
Condition2       1459 non-null category
BldgType         1459 non-null category
HouseStyle       1459 non-null category
RoofStyle        1459 non-null category
RoofMatl         1459 non-null category
Exterior1st      1459 non-null category
Exterior2nd      1459 non-null category
MasVnrType       1459 non-null category
ExterQual        1459 non-null category
ExterCond        1459 non-null category
Foundation       1459 non-null category
BsmtQual         1459 non-null category
BsmtCond         1459 non-null category
BsmtExposu

In [23]:
train_cat.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,MSSubClass
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal,60
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal,20


In [24]:
test_cat.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,MSSubClass
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,CBlock,TA,TA,No,Rec,LwQ,GasA,TA,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal,20
1462,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Y,SBrkr,Gd,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal,20


In [25]:
train_num =pd.DataFrame(train_num)
train_num.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,856.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,8.0,0.0,2003.0,2.0,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0
1,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,1262.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,6.0,1.0,1976.0,2.0,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0


In [26]:
test_num =pd.DataFrame(test_num)
test_num.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,270.0,882.0,896.0,0.0,0.0,896.0,0.0,0.0,1.0,0.0,2.0,1.0,5.0,0.0,1961.0,1.0,730.0,140.0,0.0,0.0,0.0,120.0,0.0,0.0,6.0,2010.0
1,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,406.0,1329.0,1329.0,0.0,0.0,1329.0,0.0,0.0,1.0,1.0,3.0,1.0,6.0,0.0,1958.0,1.0,312.0,393.0,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0


In [27]:
for col in train_cat:
    train_cat[col] = train_cat[col].cat.codes
for col in test_cat:
    test_cat[col] = test_cat[col].cat.codes

In [28]:
train_cat.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,MSSubClass
0,3,1,3,3,0,4,0,5,2,2,0,5,1,0,12,13,1,2,4,2,2,3,3,2,5,1,0,1,4,2,6,2,1,1,4,4,2,8,4,5
1,3,1,3,3,0,2,0,24,1,2,0,2,1,0,8,8,2,3,4,1,2,3,1,0,5,1,0,1,4,3,6,4,1,1,4,4,2,8,4,0


In [29]:
train_num.index = train_cat.index

In [30]:
test_num.index = test_cat.index

In [31]:
train_cat = pd.get_dummies(train_cat)
test_cat = pd.get_dummies(test_cat)

In [32]:
train_ = train_num.join(train_cat)

In [33]:
test_ = test_num.join(test_cat)

In [34]:
scalar = MinMaxScaler()
train_ = scalar.fit_transform(train_)
test_ = scalar.transform(test_)

In [35]:
# import xgboost
# clf=xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.07, gamma=0, subsample=0.75,
# colsample_bytree=1, max_depth=7)
# scores = cross_val_score(clf, train_, label, cv=5).mean()
# scores

In [36]:
import lightgbm as lgb
lightgbm = lgb.LGBMRegressor(objective='regression', 
                                       num_leaves=8,
                                       learning_rate=0.03, 
                                       n_estimators=4000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )
scores = cross_val_score(lightgbm, train_, label, cv=5).mean()
scores

0.9130291945249068

In [37]:
clf.fit(train_ , label)
pre = clf.predict(test_)
submit = pd.read_csv('../input/sample_submission.csv')
submit.head()


submit.SalePrice = pre

submit.to_csv('submit.csv', index = False)

In [38]:
submit.head()

,Id,SalePrice
0,1461,126495.468750
1,1462,156061.984375
2,1463,185209.500000
3,1464,189659.484375
4,1465,199411.265625
